<a href="https://colab.research.google.com/github/willystumblr/MySolar-ESS/blob/main/solar_energy_predict/Prediction_example2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash

unzip outputs.zip

In [ ]:
%%bash

unzip /content/models

In [9]:
import json

JSON = "./model_results.json"

with open(JSON, 'r') as file:
    model_results = json.load(file)

model_results    

{'LG도서관': [{'Algo': 'LSTM',
   'batch_size': 12,
   'dropout': 0.3,
   'epochs': 10,
   'lag': 24,
   'loss': {'loss': [73.24484252929688,
     28.793087005615234,
     20.564462661743164,
     18.35569953918457,
     17.56393051147461,
     16.69893455505371,
     16.353017807006836,
     16.198287963867188,
     15.718523025512695,
     15.861954689025879],
    'val_loss': [37.30930709838867,
     18.101003646850586,
     14.673748970031738,
     13.260831832885742,
     12.722332000732422,
     12.226798057556152,
     12.302059173583984,
     12.03266429901123,
     12.004964828491211,
     11.944161415100098]},
   'model_filename': '1660047217_model_24_lag.h5',
   'power_gen_rmse': 3.4553538353316497,
   'r2': 0.8764551550644374,
   'rmse': 11.93947012734114,
   'time_ran': 1660047219},
  {'Algo': 'LSTM',
   'batch_size': 12,
   'dropout': 0.3,
   'epochs': 10,
   'lag': 24,
   'loss': {'loss': [72.07402038574219,
     28.680591583251953,
     20.607763290405273,
     18.41706

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping
import time

import pandas as pd
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model
import sklearn.metrics as sklm
import numpy as np
import numpy.random as nr
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss
import math
import os
import glob

%matplotlib inline

In [10]:
model_results.keys()

dict_keys(['LG도서관', '기숙사B동', '다산빌딩', '대학C동', '동물실험동', '산업협력관', '삼성환경동', '시설관리동', '신재생에너지동', '중앙도서관', '중앙창고', '축구장', '학사과정', '학생회관'])

In [11]:
import pandas as pd

new_res_df = {}

for b in model_results:
    new_res_df[b] = pd.DataFrame(model_results[b])

In [12]:
models_by_r2 = {}
for b in model_results:
    models_by_r2[b] = new_res_df[b].sort_values(by='r2', ascending=False)

models_by_r2['학생회관']

,Algo,lag,dropout,epochs,batch_size,loss,rmse,power_gen_rmse,r2,model_filename,time_ran
1,LSTM,256,0.33,30,12,"{'loss': [29.467470169067383, 16.2115631103515...",13.475642,3.670918,0.870593,1660052167_model_256_lag.h5,1660052169
0,LSTM,24,0.30,10,12,"{'loss': [75.72154998779297, 28.83264541625976...",14.648418,3.827325,0.859331,1660047740_model_24_lag.h5,1660047741


In [14]:
model = {}
for b in model_results:
    model_filename = models_by_r2[b]['model_filename'].iloc[0]
    if(model_filename.endswith('.pkl')):
        model[b] = joblib.load('./models/'+ b +"/"+ model_filename)
    else:
        model[b] = load_model('./models/'+b+"/"+ model_filename)

In [25]:
PATH = "/content/OBS_ASOS_TIM_20220810233900.csv"
df_forecast = pd.read_csv(PATH, encoding="euc_kr")
df_forecast.head()

,지점,지점명,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),일조(hr),일사(MJ/m2),전운량(10분위),지면온도(°C)
0,156,광주,2022-07-29 00:00,27.4,NaN,0.8,94,34.2,26.3,1004.2,NaN,NaN,2,26.7
1,156,광주,2022-07-29 01:00,26.9,NaN,0.6,96,34.0,26.2,1004.3,NaN,NaN,2,26.3
2,156,광주,2022-07-29 02:00,26.5,NaN,1.0,98,33.8,26.1,1003.9,NaN,NaN,2,25.9
3,156,광주,2022-07-29 03:00,26.3,NaN,1.2,98,33.4,25.9,1004.1,NaN,NaN,2,25.6
4,156,광주,2022-07-29 04:00,26.3,NaN,0.2,97,33.0,25.7,1004.1,NaN,NaN,2,25.2


In [26]:
df_forecast = df_forecast.drop(axis=1, columns=['지점', '지점명'])
df_forecast["일시"] = pd.to_datetime(df_forecast["일시"])

In [27]:
df_forecast

,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),일조(hr),일사(MJ/m2),전운량(10분위),지면온도(°C)
0,2022-07-29 00:00:00,27.4,NaN,0.8,94,34.2,26.3,1004.2,NaN,NaN,2,26.7
1,2022-07-29 01:00:00,26.9,NaN,0.6,96,34.0,26.2,1004.3,NaN,NaN,2,26.3
2,2022-07-29 02:00:00,26.5,NaN,1.0,98,33.8,26.1,1003.9,NaN,NaN,2,25.9
3,2022-07-29 03:00:00,26.3,NaN,1.2,98,33.4,25.9,1004.1,NaN,NaN,2,25.6
4,2022-07-29 04:00:00,26.3,NaN,0.2,97,33.0,25.7,1004.1,NaN,NaN,2,25.2
5,2022-07-29 05:00:00,25.8,NaN,1.0,99,32.8,25.6,1004.3,NaN,NaN,2,24.9
6,2022-07-29 06:00:00,25.6,NaN,0.8,99,32.4,25.4,1004.4,0.0,0.06,0,24.8
7,2022-07-29 07:00:00,26.2,NaN,1.4,98,33.2,25.8,1004.4,0.8,0.40,0,25.5
8,2022-07-29 08:00:00,27.9,NaN,2.5,91,34.0,26.2,1004.3,1.0,1.08,2,29.5
9,2022-07-29 09:00:00,30.2,NaN,1.3,80,34.2,26.3,1004.3,1.0,1.89,3,36.2


In [28]:
df_forecast["일사(MJ/m2)"].fillna(0.0, inplace=True)
df_forecast["강수량(mm)"].fillna(0.0, inplace=True)
df_forecast["일조(hr)"].fillna(0.0, inplace=True)

In [29]:
df_forecast = df_forecast.assign(Cloud = lambda x: x['전운량(10분위)'] >= 5)

In [33]:
df_forecast = df_forecast.assign(Hour = lambda x: x['일시'].dt.hour)
df_forecast = df_forecast.assign(Month = lambda x: x['일시'].dt.month)
df_forecast = df_forecast.assign(Year = lambda x: x['일시'].dt.year)
df_forecast = df_forecast.assign(Day = lambda x: x['일시'].dt.day)

In [34]:
print(df_forecast['Hour'].unique())
Features_forecast_total = df_forecast['Hour']
enc = preprocessing.LabelEncoder()
enc.fit(Features_forecast_total)
Features_forecast_total = enc.transform(Features_forecast_total)
print(Features_forecast_total)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]


In [35]:
ohe = preprocessing.OneHotEncoder()
encoded = ohe.fit(Features_forecast_total.reshape(-1,1))
print(Features_forecast_total.reshape(-1,1))
Features_forecast_total = encoded.transform(Features_forecast_total.reshape(-1,1)).toarray()
Features_forecast_total[:10,:]

[[ 0]
 [ 1]
 [ 2]
 [ 3]
 [ 4]
 [ 5]
 [ 6]
 [ 7]
 [ 8]
 [ 9]
 [10]
 [11]
 [12]
 [13]
 [14]
 [15]
 [16]
 [17]
 [18]
 [19]
 [20]
 [21]
 [22]
 [23]]


array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 

In [36]:
def encode_string(cat_feature):
    ## First encode the strings to numeric categories
    enc = preprocessing.LabelEncoder()
    enc.fit(cat_feature)
    enc_cat_feature = enc.transform(cat_feature)
    ## Now, apply one hot encoding
    ohe = preprocessing.OneHotEncoder()
    encoded = ohe.fit(enc_cat_feature.reshape(-1,1))
    return encoded.transform(enc_cat_feature.reshape(-1,1)).toarray()
    

categorical_columns = ['Cloud']

for col in categorical_columns:
    temp = encode_string(df_forecast[col])
    Features_forecast_total = np.concatenate([Features_forecast_total, temp], axis = 1)
    #print(Features)

print(Features_forecast_total.shape)
print(Features_forecast_total[:2,:])  

(24, 26)
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  1. 0.]]


In [37]:
Features_forecast_total = np.concatenate([Features_forecast_total, np.array(df_forecast[['기온(°C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '증기압(hPa)', '이슬점온도(°C)',
       '일조(hr)', '일사(MJ/m2)', 'Hour', 'Month', 'Day']])], axis = 1)
Features_forecast_total[:2,:]

array([[ 1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  1. ,  0. , 27.4,  0. ,  0.8, 94. , 34.2, 26.3,  0. ,
         0. ,  0. ,  7. , 29. ],
       [ 0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  1. ,  0. , 26.9,  0. ,  0.6, 96. , 34. , 26.2,  0. ,
         0. ,  1. ,  7. , 29. ]])

In [38]:
Features_forecast_total.shape

(24, 37)

In [39]:
scaler = preprocessing.StandardScaler().fit(Features_forecast_total[:,10:])
Features_forecast_total[:,10:] = scaler.transform(Features_forecast_total[:,10:])
print(Features_forecast_total.shape)
Features_forecast_total[:10,:]

(24, 37)


array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        -0.20851441, -0.20851441, -0.20851441, -0.20851441, -0.20851441,
        -0.20851441, -0.20851441, -0.20851441, -0.20851441, -0.20851441,
        -0.20851441, -0.20851441, -0.20851441, -0.20851441,  0.51298918,
        -0.51298918, -0.88549033,  0.        , -1.1405743 ,  1.05225962,
         1.30183541,  1.28113866, -1.04938139, -0.90482963, -1.66132477,
         0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        -0.20851441, -0.20851441, -0.20851441, -0.20851441, -0.20851441,
        -0.20851441, -0.20851441, -0.20851441, -0.20851441, -0.20851441,
        -0.20851441, -0.20851441, -0.20851441, -0.20851441,  0.51298918,
        -0.51298918, -1.04624501,  0.        , -1.52342441,  1.17515126,
         1.18073

In [40]:
Features_forecast_total = Features_forecast_total.reshape((Features_forecast_total.shape[0], 1, Features_forecast_total.shape[1]))
    # make a prediction

res = {}

for d in model_results:
    res[d] = model[d].predict(Features_forecast_total)
    # print(res)
    # sets floor for res
    res[d] = [x[0] if x > 0 else 0 for x in res[d]]

In [41]:
df_total = {}

for d in model_results:
    df_total[d] = pd.concat([df_forecast, pd.DataFrame(res[d], columns=['predicted_power'], index=df_forecast.index)], axis=1)

In [45]:
df_total['학생회관']

,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),일조(hr),일사(MJ/m2),전운량(10분위),지면온도(°C),Cloud,Hour,Month,Year,Day,predicted_power
0,2022-07-29 00:00:00,27.4,0.0,0.8,94,34.2,26.3,1004.2,0.0,0.00,2,26.7,False,0,7,2022,29,0.000000
1,2022-07-29 01:00:00,26.9,0.0,0.6,96,34.0,26.2,1004.3,0.0,0.00,2,26.3,False,1,7,2022,29,0.000000
2,2022-07-29 02:00:00,26.5,0.0,1.0,98,33.8,26.1,1003.9,0.0,0.00,2,25.9,False,2,7,2022,29,0.000000
3,2022-07-29 03:00:00,26.3,0.0,1.2,98,33.4,25.9,1004.1,0.0,0.00,2,25.6,False,3,7,2022,29,0.000000
4,2022-07-29 04:00:00,26.3,0.0,0.2,97,33.0,25.7,1004.1,0.0,0.00,2,25.2,False,4,7,2022,29,0.000000
5,2022-07-29 05:00:00,25.8,0.0,1.0,99,32.8,25.6,1004.3,0.0,0.00,2,24.9,False,5,7,2022,29,0.000000
6,2022-07-29 06:00:00,25.6,0.0,0.8,99,32.4,25.4,1004.4,0.0,0.06,0,24.8,False,6,7,2022,29,0.000000
7,2022-07-29 07:00:00,26.2,0.0,1.4,98,33.2,25.8,1004.4,0.8,0.40,0,25.5,False,7,7,2022,29,0.000000
8,2022-07-29 08:00:00,27.9,0.0,2.5,91,34.0,26.2,1004.3,1.0,1.08,2,29.5,False,8,7,2022,29,8.640085
9,2022-07-29 09:00:00,30.2,0.0,1.3,80,34.2,26.3,1004.3,1.0,1.89,3,36.2,False,9,7,2022,29,15.522281


In [47]:
import os

for pred in df_total:
    os.makedirs("./outputs_2/"+pred)

for pred in df_total:
    df_total[pred].to_csv("./outputs_2/"+pred+"/prediction.csv")

In [ ]:
%%bash
zip -r outputs_2.zip outputs_2 